# Step-1: Loading the Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import glob 
filePaths = glob.glob("/home/prakhar/Desktop/College/3rdYear/SEM-VI/MinorProject/Datasets/archive-2/*.csv")
df_list = [pd.read_csv(file,low_memory=False) for file in filePaths]
df=pd.concat(df_list,ignore_index=True)
print(df.shape)
print(df.head())

(2830743, 79)
   Destination Port  Flow Duration  ...  Idle Min   Label
0             54865              3  ...         0  BENIGN
1             55054            109  ...         0  BENIGN
2             55055             52  ...         0  BENIGN
3             46236             34  ...         0  BENIGN
4             54863              3  ...         0  BENIGN

[5 rows x 79 columns]


In [3]:
df.columns = df.columns.str.strip()
df['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

# Step-2: Pre-Processing

In [4]:
df.dtypes #Viewing Datatype

Destination Port                 int64
Flow Duration                    int64
Total Fwd Packets                int64
Total Backward Packets           int64
Total Length of Fwd Packets      int64
                                ...   
Idle Mean                      float64
Idle Std                       float64
Idle Max                         int64
Idle Min                         int64
Label                           object
Length: 79, dtype: object

In [5]:
is_inf = df.isin([np.inf, -np.inf])

inf_counts = is_inf.sum()

# Filter to show only columns with infinite values
columns_with_inf = inf_counts[inf_counts > 0]
print("Columns with infinite values:\n", columns_with_inf)

nan_counts = df.isnull().sum()

# Filter to show only columns with NaN values
columns_with_nan = nan_counts[nan_counts > 0]
print("\nColumns with NaN values:\n", columns_with_nan)

combined_counts = nan_counts + inf_counts

# Filter to show only columns with any NaN or inf values
columns_with_nan_or_inf = combined_counts[combined_counts > 0]
print("\nColumns with NaN or infinite values:\n", columns_with_nan_or_inf)

duplicate_rows_count = df.duplicated().sum()
print("\nNumber of duplicate rows:", duplicate_rows_count)

Columns with infinite values:
 Flow Bytes/s      1509
Flow Packets/s    2867
dtype: int64

Columns with NaN values:
 Flow Bytes/s    1358
dtype: int64

Columns with NaN or infinite values:
 Flow Bytes/s      2867
Flow Packets/s    2867
dtype: int64

Number of duplicate rows: 308381


In [6]:
df.replace([np.inf, -np.inf], np.nan, inplace=True) #Replacing inf values
df.dropna(inplace=True) #Replacing NaN values
df.drop_duplicates(inplace=True) #Removing duplicates to improve accuracy
category_mapping = {
    'BENIGN': 'BENIGN',
    'Bot': 'BOTNET',
    'DDoS': 'DOS',
    'DoS GoldenEye': 'DOS',
    'DoS Hulk': 'DOS',
    'DoS Slowhttptest': 'DOS',
    'DoS slowloris': 'DOS',
    'FTP-Patator': 'BRUTE_FORCE',
    'SSH-Patator': 'BRUTE_FORCE',
    'Heartbleed': 'WEB_ATTACK',
    'Infiltration': 'WEB_ATTACK',
    'PortScan': 'RECONNAISSANCE',
    'Web Attack � Brute Force': 'WEB_ATTACK',
    'Web Attack � Sql Injection': 'WEB_ATTACK',
    'Web Attack � XSS': 'WEB_ATTACK'
}
#Applying Mapping to simplify labels
df['Label'] =df['Label'].map(category_mapping)

In [7]:
print(df['Label'].value_counts())  # Before encoding

Label
BENIGN            2095057
DOS                321759
RECONNAISSANCE      90694
BRUTE_FORCE          9150
WEB_ATTACK           2190
BOTNET               1948
Name: count, dtype: int64


In [8]:
from sklearn.preprocessing import LabelEncoder #After Encoding
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

In [9]:
print(df['Label'].value_counts())  # Before encoding

Label
0    2095057
3     321759
4      90694
2       9150
5       2190
1       1948
Name: count, dtype: int64


In [10]:
import pandas as pd

# Get class counts
class_counts = df['Label'].value_counts()

# Select classes with more than 5000 samples
selected_classes = class_counts[class_counts > 50000].index.tolist()

# Manually include "WEB_ATTACK" and "BOTNET" if they exist in the dataset
for attack in [1, 5, 2]:
    if attack in df['Label'].values and attack not in selected_classes:
        selected_classes.append(attack)

# Select rows where 'Label' is in selected_classes
selected = df[df['Label'].isin(selected_classes)]

dfs = []
for name in selected_classes:
    df_class = selected[selected['Label'] == name]

    # Downsample if the class has more than 5000 samples
    if len(df_class) > 50000:
        df_class = df_class.sample(n=50000, random_state=0)

    dfs.append(df_class)

# Combine all sampled data
balanced_df = pd.concat(dfs, ignore_index=True)

# Check final class distribution
print(balanced_df['Label'].value_counts())

Label
0    50000
3    50000
4    50000
2     9150
5     2190
1     1948
Name: count, dtype: int64


In [11]:
balanced_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,53,62874,4,2,116,186,29,29,29.000,0.000000,93,93,93.000000,0.000000,4803.257308,95.428953,1.257480e+04,1.721929e+04,31970,2,30902,1.030067e+04,1.783608e+04,30896,3,2,2.000,0.000000e+00,2,2,0,0,0,0,80,40,63.619302,31.809651,29,93,47.285714,31.228802,975.238095,0,0,0,0,0,0,0,0,0,55.166667,29.000,93.000000,80,0,0,0,0,0,0,4,116,2,186,-1,-1,3,20,0.00000,0.00000,0,0,0.000,0.00000,0,0,0
1,57225,86,1,1,0,0,0,0,0.000,0.000000,0,0,0.000000,0.000000,0.000000,23255.813950,8.600000e+01,0.000000e+00,86,86,0,0.000000e+00,0.000000e+00,0,0,0,0.000,0.000000e+00,0,0,0,0,0,0,32,32,11627.906980,11627.906980,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,1,0.000000,0.000,0.000000,32,0,0,0,0,0,0,1,0,1,0,905,229,0,32,0.00000,0.00000,0,0,0.000,0.00000,0,0,0
2,80,116003861,16,14,440,412,356,0,27.500,87.612784,340,0,29.428571,89.402903,7.344583,0.258612,4.000133e+06,4.874562e+06,10000000,28,116000000,7.733591e+06,4.140718e+06,10000000,15994,110000000,8497084.154,3.759107e+06,10000000,999,0,0,0,0,332,288,0.137926,0.120686,0,356,27.483871,85.606803,7328.524731,0,0,0,1,0,0,0,0,0,28.400000,27.500,29.428571,332,0,0,0,0,0,0,16,440,14,412,8192,4736,15,20,46441.09091,31255.09835,140678,36852,9995557.727,14892.00092,10000000,9950679,0
3,443,5683083,8,6,397,3716,216,0,49.625,79.224703,1460,0,619.333333,679.909014,723.726893,2.463452,4.371602e+05,1.496191e+06,5413681,3,5683083,8.118690e+05,2.030700e+06,5413681,3,52315,10463.000,1.314115e+04,27091,48,0,0,0,0,172,132,1.407687,1.055765,0,1460,274.200000,503.464880,253476.885700,0,0,0,1,0,0,0,0,0,293.785714,49.625,619.333333,172,0,0,0,0,0,0,8,397,6,3716,8192,119,7,20,269399.00000,0.00000,269399,269399,5413681.000,0.00000,5413681,5413681,0
4,53,23652,1,1,48,128,48,48,48.000,0.000000,128,128,128.000000,0.000000,7441.231186,84.559445,2.365200e+04,0.000000e+00,23652,23652,0,0.000000e+00,0.000000e+00,0,0,0,0.000,0.000000e+00,0,0,0,0,0,0,20,20,42.279723,42.279723,48,128,74.666667,46.188022,2133.333333,0,0,0,0,0,0,0,0,1,112.000000,48.000,128.000000,20,0,0,0,0,0,0,1,48,1,128,-1,-1,0,20,0.00000,0.00000,0,0,0.000,0.00000,0,0,0


In [12]:
from imblearn.over_sampling import SMOTE

x = balanced_df.drop('Label', axis=1)
y = balanced_df['Label']

smote = SMOTE(sampling_strategy='auto', random_state=0)
X_upsampled, y_upsampled = smote.fit_resample(x, y)

blnc_data = pd.DataFrame(X_upsampled)
blnc_data['Label'] = y_upsampled
blnc_data = blnc_data.sample(frac=1)

print(blnc_data['Label'].value_counts())
blnc_data.head()

Label
2    50000
3    50000
0    50000
5    50000
4    50000
1    50000
Name: count, dtype: int64


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
213333,21,8516157,9,15,115,188,26,0,12.886869,10.868268,34,0,12.533333,14.549750,3.569412e+01,2.818175,3.702677e+05,9.620163e+05,3087266,2,5696275,7.120345e+05,1.322438e+06,3133463,301,8516093,6.082924e+05,1.187551e+06,3087266,2,0,0,0,0,296,488,1.056816,1.761359,0,34,12.159273,13.022663,1.696497e+02,0,0,0,1,0,0,0,0,1,12.665909,12.886869,12.533333,296,0,0,0,0,0,0,9,115,15,188,29200,227,6,32,0.0,0.0,0,0,0.0,0.0,0,0,2
212423,21,8754891,9,15,119,188,27,0,13.317423,11.288208,34,0,12.533333,14.549750,3.516396e+01,2.741325,3.806475e+05,9.955602e+05,3315745,3,6319480,7.899351e+05,1.463104e+06,3359860,259,8754776,6.253412e+05,1.229856e+06,3315745,5,0,0,0,0,296,488,1.027997,1.713328,0,34,12.314272,13.141209,1.726918e+02,0,0,0,1,0,0,0,0,1,12.827367,13.317423,12.533333,296,0,0,0,0,0,0,9,119,15,188,29200,227,6,32,0.0,0.0,0,0,0.0,0.0,0,0,2
64523,80,101487893,12,7,1600,11595,320,0,133.333333,164.777168,5792,0,1656.428571,2118.227235,1.300155e+02,0.187214,5.638216e+06,2.350000e+07,99800000,0,101000000,9.226073e+06,3.000000e+07,99800000,0,1672392,2.787320e+05,6.822486e+05,1671366,2,0,0,0,0,392,232,0.118241,0.068974,0,5792,659.750000,1412.988765,1.996537e+06,0,0,0,0,1,0,0,0,0,694.473684,133.333333,1656.428571,392,0,0,0,0,0,0,12,1600,7,11595,274,235,5,32,1054.0,0.0,1054,1054,99800000.0,0.0,99800000,99800000,3
51788,80,98671866,5,8,410,11595,398,0,82.000000,176.674843,4344,0,1449.375000,1729.380148,1.216659e+02,0.131750,8.222656e+06,2.850000e+07,98600000,5,98700000,2.470000e+07,4.930000e+07,98600000,5,98700000,1.410000e+07,3.730000e+07,98600000,50,0,0,0,0,144,264,0.050673,0.081077,0,4344,857.928571,1456.938965,2.122671e+06,1,0,0,0,0,0,0,0,1,923.923077,82.000000,1449.375000,144,0,0,0,0,0,0,5,410,8,11595,0,235,2,20,13981.0,0.0,13981,13981,98600000.0,0.0,98600000,98600000,3
12035,53,254,2,2,60,264,30,30,30.000000,0.000000,132,132,132.000000,0.000000,1.275591e+06,15748.031500,8.466667e+01,1.047107e+02,203,4,47,4.700000e+01,0.000000e+00,47,47,4,4.000000e+00,0.000000e+00,4,4,0,0,0,0,64,64,7874.015748,7874.015748,30,132,70.800000,55.867701,3.121200e+03,0,0,0,0,0,0,0,0,1,88.500000,30.000000,132.000000,64,0,0,0,0,0,0,2,60,2,264,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0,0


In [13]:
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
x = blnc_data.drop('Label',axis=1)
y = blnc_data['Label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)
n_components = 20
ipca = IncrementalPCA(n_components=n_components, batch_size=500)
x_pca = ipca.fit_transform(X_scaled) 
print(f"Explained Variance Ratio: {sum(ipca.explained_variance_ratio_):.4f}")
x = x_pca

Explained Variance Ratio: 0.9331


In [14]:
pca_df = pd.DataFrame(x, columns=[f'PC{i+1}' for i in range(x.shape[1])])
print(pca_df)

              PC1       PC2       PC3  ...      PC18      PC19      PC20
0       -1.545303  0.058455 -0.126318  ...  0.486768  0.413953 -0.023742
1       -1.523088  0.056606 -0.139072  ...  0.490652  0.414715 -0.022617
2       10.049724 -0.934680 -0.219918  ... -0.052286  0.436982  1.169230
3       13.925069 -1.536022 -4.063449  ... -0.794591 -0.266956 -0.403089
4       -1.604788 -0.112223 -0.239975  ...  0.117388 -0.032719 -0.011579
...           ...       ...       ...  ...       ...       ...       ...
299995  -1.943023 -0.071679 -0.036009  ...  0.317237  0.372677 -0.151185
299996  -1.803018 -0.052848 -0.086841  ...  0.386884  0.464821 -0.124440
299997  -1.850144 -0.186220 -0.589143  ... -0.605085  0.273834  0.012560
299998   0.599252  0.539964  1.976786  ...  0.041944  0.210577 -0.224510
299999  -1.502719 -0.126473 -0.492230  ... -0.891934 -0.561011  0.010110

[300000 rows x 20 columns]


In [15]:
pca_df.shape

(300000, 20)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,shuffle=True)

# Step-3: Applying Classifier Algorithms

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import RocCurveDisplay
import tensorflow as tf
import keras
from keras import layers,regularizers,optimizers

#### Step-3.1: Applying Random Foressts

In [ ]:
from sklearn.ensemble import RandomForestClassifier #Importing Random Forest classifier
rf = RandomForestClassifier(n_estimators=1,max_depth=7,random_state=42)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test) #Performing Random Forest

In [ ]:
print(classification_report(y_test, y_pred)) #Printing Classification Report
class_indices = rf.classes_
print(class_indices)

In [ ]:
cv_rf = cross_val_score(rf, x_train, y_train, cv=5)
print(f'\nCross-validation scores:', ', '.join(map(str, cv_rf)))
print(f'\nMean cross-validation score: {cv_rf.mean():.2f}')

In [ ]:
print(rf.score(x_train,y_train)) #RF Score x_train,y_train

In [ ]:
print(rf.score(x_test, y_test)) #RF Score x_test,y_test

In [ ]:
# Compute ROC curve
# False Positive Rate, True Positive Rate
conf_mtr = confusion_matrix(y_true=y_test,y_pred=y_pred)
# Creating a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mtr, annot=True, cmap='Blues',
            xticklabels=rf.classes_, yticklabels=rf.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()
plt.show()

In [ ]:
lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
for i, class_name in enumerate(lb.classes_):
    y_score = rf.predict_proba(x_test)[:, i]
    RocCurveDisplay.from_predictions(
        y_test_bin[:, i],
        y_score,
        name=f'ROC curve for {class_name}',
        ax=plt.gca()
    )

plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

#### Step-3.2: Applying KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier #Importing KNN
knn = KNeighborsClassifier(n_neighbors= 100) #Changing Value Manually (Due to Performance Reasons)

knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)  # Performing Random Forest

In [ ]:
print(classification_report(y_test, y_pred)) #Printing Classification Report

In [ ]:
cv_knn = cross_val_score(knn, x_train, y_train, cv=5)
print(f'\nCross-validation scores:', ', '.join(map(str, cv_knn)))
print(f'\nMean cross-validation score: {cv_knn.mean():.2f}')

In [ ]:
print(knn.score(x_train, y_train)) #RF Score x_train,y_train

In [ ]:
print(knn.score(x_test, y_test)) #RF Score x_train,y_train

In [ ]:
# Compute Confusion Matrix
# False Positive Rate, True Positive Rate
conf_mtr = confusion_matrix(y_true=y_test, y_pred=y_pred)
# Creating a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mtr, annot=True, cmap='Blues',
            xticklabels=rf.classes_, yticklabels=rf.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.tight_layout()
plt.show()

In [ ]:
lb = LabelBinarizer()
y_test_bin = lb.fit_transform(y_test)

# Plot ROC curve for each class
plt.figure(figsize=(10, 8))
for i, class_name in enumerate(lb.classes_):
    y_score = rf.predict_proba(x_test)[:, i]
    RocCurveDisplay.from_predictions(
        y_test_bin[:, i],
        y_score,
        name=f'ROC curve for {class_name}',
        ax=plt.gca()
    )

plt.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

#### Step-3.3: Applying DNN

In [22]:
model = keras.Sequential()
model.add(layers.Input(shape=(x_train.shape[1],)))  # Input layer

model.add(layers.Dense(128, activation='relu'))  # Hidden layer
model.add(layers.Dropout(0.1))  # Dropout layer

model.add(layers.Dense(256, activation='relu'))  # Hidden layer
model.add(layers.Dropout(0.1))  # Dropout layer

model.add(layers.Dense(128, activation='relu'))  # Hidden layer
model.add(layers.Dropout(0.1))  # Dropout layer

model.add(layers.Dense(6, activation='softmax'))  # Output layer

In [23]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 128)            │         2,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,382 (271.02 KB)

 Trainable params: 69,382 (271.02 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy']) #compiling model


In [ ]:
x_val = x_train[:5000]
partial_x_train = x_train[5000:]

y_val = y_train[:5000]
partial_y_train = y_train[5000:]

model.fit(partial_x_train,partial_y_train,epochs=10,batch_size=16,validation_data=(x_val,y_val)) #Fitting Model
model.save("DNN.keras") #Saving the model

Epoch 1/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.8797 - loss: 0.4159 - val_accuracy: 0.9642 - val_loss: 0.1212
Epoch 2/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9627 - loss: 0.1248 - val_accuracy: 0.9700 - val_loss: 0.0962
Epoch 3/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9694 - loss: 0.1022 - val_accuracy: 0.9728 - val_loss: 0.0838
Epoch 4/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9732 - loss: 0.0883 - val_accuracy: 0.9764 - val_loss: 0.0764
Epoch 5/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - accuracy: 0.9753 - loss: 0.0835 - val_accuracy: 0.9758 - val_loss: 0.0761
Epoch 6/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9760 - loss: 0.0795 - val_accuracy: 0.9768 - val_loss: 0.0763
Epoch 7/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9773 - loss: 0.0742 - val_accuracy: 0.9790 - val_loss: 0.0725
Epoch 8/10
13750/13750 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 

In [27]:
print(f"Results of Testing") #Testing the model
model.evaluate(x_test,y_test)

Results of Testing
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 2s 958us/step - accuracy: 0.9815 - loss: 0.0598


[0.06166059151291847, 0.9807866811752319]

In [28]:
print(f"Result of Prediction") #Performing Predictions
predictions=model.predict(x=x_test[:5])
print(predictions)
print("predictions shape:", predictions.shape)

Result of Prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
[[1.33091249e-04 9.99861836e-01 6.52369522e-07 2.05834021e-06
  2.16994954e-06 2.65824042e-07]
 [1.58503997e-06 2.62882781e-26 5.78740398e-12 9.99998450e-01
  5.74835042e-08 3.00692041e-11]
 [4.57531996e-06 1.51241053e-12 9.99995470e-01 8.66247873e-09
  5.90507476e-10 1.92845331e-08]
 [9.75697840e-05 9.99902368e-01 1.19781957e-20 1.20948046e-15
  1.21502725e-19 8.58095377e-23]
 [1.46648614e-03 2.61060509e-12 9.98411536e-01 1.19408534e-04
  7.77039418e-07 1.78240521e-06]]
predictions shape: (5, 6)


#### Step-3.4: Applying DCNN

In [ ]:
model_cnn = keras.Sequential()
model_cnn.add(layers.Input(shape=(x_train.shape[1],)))  # Input Layer

model_cnn.add(layers.Conv1D(128, 20, data_format='last',
              activation='relu', padding='same'))  # Hidden Layer
model_cnn.add(layers.Conv1D(256, 20, data_format='last',
              activation='relu', padding='same'))  # Hidden Layer

model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(256, activation='relu',
          kernel_regularizer=keras.regularizers.l1_l2()))

model_cnn.add(layers.Dense(6,activation='softmax'))

In [ ]:
model_cnn.summary()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model_cnn.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # compiling model

In [ ]:
x_val = x_train[:5000]
partial_x_train = x_train[5000:]

y_val = y_train[:5000]
partial_y_train = y_train[5000:]

model_cnn.fit(partial_x_train, partial_y_train, epochs=10, batch_size=16,
          validation_data=(x_val, y_val))  # Fitting Model
model_cnn.save("CNN.keras")  # Saving the model

In [ ]:
print(f"Results of Testing")  # Testing the model
model_cnn.evaluate(x_test, y_test)

In [ ]:
print(f"Result of Prediction")  # Performing Predictions
predictions = model_cnn.predict(x=x_test[:5])
print(predictions)
print("predictions shape:", predictions.shape)